# Toronto Neighborhood
---

Install/import libraries used

In [15]:
#!pip install bs4
#!pip install requests
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd

### Web Scraping:

Wikipedia holds a table of all the relevant postal code data. Set url to table address

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

Read in HTML from url using beautiful soup. (install the parser used) in this case, we'll use the python parser to read in the table

In [13]:
#!pip install html5lib

data  = requests.get(url).text
soup = BeautifulSoup(data,"html.parser")

Read in the contents into a meaningful dataframe. Entries without a borough assgined are dropped. Postal codes with multiple neighborhoods are concathenated into one entry. 

In [16]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

Verify dimensions of the dataset.

In [53]:
df.shape

(103, 3)

### Geospatial Data:

Using the geocoder api, grab the (lat,lng) coordinates of each neighborhood.

In [ ]:
#!pip install geocoder
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format("M5G"))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

If geocoder isn't working, use the geospatial data from the .csv file.

Rename the postal code columns and merge the two dataframes together, matched on the postal code.

In [125]:
geo_data = pd.read_csv("Geospatial_Coordinates.csv")
geo_data.rename(columns={"Postal Code": "PostalCode"}, inplace=True)

toronto_borough = df.merge(geo_data, on=["PostalCode"])
toronto_borough = toronto_borough[toronto_borough['Borough'].str.contains('Toronto', na=False)].reset_index(drop=True)


In [126]:
toronto_borough.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
9,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106


In [94]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [76]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [95]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(toronto_borough['Latitude'], toronto_borough['Longitude'], toronto_borough['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Cluster

In [127]:
toronto_borough.groupby('Borough').count()

,PostalCode,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,17,17,17,17
Downtown Toronto Stn A,1,1,1,1
East Toronto,4,4,4,4
East Toronto Business,1,1,1,1
East York/East Toronto,1,1,1,1
West Toronto,6,6,6,6


In [142]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_borough[['Neighborhood','PostalCode']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = toronto_borough['Borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [143]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped

,Borough,Berczy Park,"Brockton, Parkdale Village, Exhibition Place","CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",Central Bay Street,Christie,Church and Wellesley,"Commerce Court, Victoria Hotel",Davisville,Davisville North,...,M5W,M5X,M6G,M6H,M6J,M6K,M6P,M6R,M6S,M7Y
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.111111,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,Downtown Toronto,0.058824,0.000000,0.058824,0.058824,0.058824,0.058824,0.058824,0.000000,0.000000,...,0.0,0.058824,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,Downtown Toronto Stn A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,East Toronto Business,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
5,East York/East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
6,West Toronto,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.0


In [133]:
# import k-means from clustering stage
#!pip install sklearn
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

/anaconda3/envs/DataScience/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


array([4, 4, 3, 1, 0, 2, 4], dtype=int32)

In [134]:
# add clustering labels
toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

#toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
#toronoto_merged = toronto_merged.join(boroughs_venues_sorted.set_index('Borough'), on='Borough')

#toronto_merged.head() # check the last columns!

In [140]:
label_col = toronto_grouped[["Borough","Cluster Labels"]]
toronto_with_labels = label_col.merge(toronto_borough, on=["Borough"])

In [141]:
toronto_with_labels.head()

,Borough,Cluster Labels,PostalCode,Neighborhood,Latitude,Longitude
0,Central Toronto,4,M4N,Lawrence Park,43.728020,-79.388790
1,Central Toronto,4,M5N,Roselawn,43.711695,-79.416936
2,Central Toronto,4,M4P,Davisville North,43.712751,-79.390197
3,Central Toronto,4,M5P,Forest Hill North & West,43.696948,-79.411307
4,Central Toronto,4,M4R,North Toronto West,43.715383,-79.405678
